In [1]:
import praw
import datetime
import csv
import pickle
import os

In [2]:
visited = set()
if os.path.isfile('visited_comments.txt'):
    with open("visited_comments.txt", 'r') as infile:
        for line in infile:
            visited.add(line.strip())
print(len(visited))

0


In [5]:
user_agent = "Test/0.1 by maxoboe"
with open('account_info.txt', 'r') as file:
    lines = file.read().splitlines()
    username = lines[0]
    password = lines[1]
    clientID = lines[2]
    secretID = lines[3]
reddit = praw.Reddit(client_id=clientID,
                     client_secret=secretID,
                     user_agent=user_agent)

with open('all_submissions.csv', 'r', encoding='utf-8') as infile, \
    open('doc2vec/all_comments.csv', 'a+', encoding='utf-8', newline='') as outfile, \
    open('doc2vec/all_submissions.csv', 'a+', encoding='utf-8', newline='') as outfile_2:
    reader = csv.DictReader(infile)
    fieldnames = ['title', 'selftext', 'ups', 'downs', 'views', 'date', 'time', 'comment', 'c_ups', 'c_downs']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    fieldnames_2 = ['title', 'selftext', 'ups', 'downs', 'views', 'date', 'time']
    writer_2 = csv.DictWriter(outfile_2, fieldnames=fieldnames_2)
    writer.writeheader()
    writer_2.writeheader()
    for row in reader:
        title = row['title'].lower()
        if '[removed]' in title or '[deleted]' in title: continue
        selftext = row['selftext'].lower()
        if '[removed]' in selftext or '[deleted]' in selftext: continue
        if row['id'] in visited: continue
        submission = reddit.submission(id=row['id'])
        views = submission.view_count
        if row['comments'] is '0':
            continue
        try:
            submission.comments.replace_more(limit=None)
            for top_level_comment in submission.comments:
                dateTime = datetime.datetime.fromtimestamp(top_level_comment.created)
                writer.writerow({'title':submission.title, 'selftext': row['selftext'], 'ups': row['ups'], 'downs': row['downs'],
                                 'views': views, 'date': dateTime.date(), 'time': dateTime.time(), 
                                 'comment': top_level_comment.body, 'c_ups': top_level_comment.ups, 
                                 'c_downs': top_level_comment.downs})
            dateTime = datetime.datetime.fromtimestamp(submission.created)
            writer_2.writerow({'title':submission.title, 'selftext': row['selftext'], 'ups': row['ups'], 'downs': row['downs'],
                                 'views': views, 'date': dateTime.date(), 'time': dateTime.time()})
            visited.add(row['id'])
            with open('visited_comments.txt', 'a+') as countfile:
                countfile.write(row['id'] + '\n')            
        except:
            print('Execution failed on ' + row['title'])
        break

In [26]:
import csv 
with open('all_comments.csv', 'r', encoding='utf-8') as infile, \
    open('processed_comments.csv', 'w+', encoding='utf-8', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['title', 'selftext', 'ups', 'downs', 'date', 'time', 'comment', 'c_ups', 'c_downs']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in reader:
        if row['comment'] == '[deleted]': continue
        if row['comment'] == '[removed]': continue
        if 'This message was automatic' in row['comment']: continue
        if 'Thank you for your submission! Unfortunately' in row['comment']: continue
        writer.writerow(row)


In [6]:
print(submission)

6qhm6


In [10]:
submission = reddit.submission(id='5rdmyx')
print(submission.view_count)

None
